In [7]:
import glob, h5py, os
import numpy as np
import matplotlib.pyplot as plt

from oc_ica import gabor_fit, plotting

In [5]:
folder = '/home/jesse/Development/results/oc_ica/nat_images_mixtures-64_sources-128/'

In [15]:
files = glob.glob(os.path.join(folder, '*.h5'))

for fname in files:
    with h5py.File(fname) as f:
        w = f['W_fits'].value
        params = plotting.get_Gabor_params(w)
        print params.shape
    break

0.0 4.71238898038 0.525077115546 0.51908373559
0.448798950513 4.71238898038 0.520953092692 0.518931369178
0.897597901026 4.71238898038 0.543945201844 0.518862987664
1.34639685154 4.71238898038 0.52890989241 0.518528450792
1.79519580205 4.71238898038 0.534539796933 0.518501920651
2.24399475256 4.71238898038 0.549312081604 0.518495491339
2.69279370308 4.71238898038 0.533778419728 0.518494943763
3.14159265359 4.71238898038 0.529147735106 0.518491675185
0.0 4.71238898038 0.434486867169 0.384022584411
0.448798950513 4.71238898038 0.425347768818 0.384014984113
0.897597901026 4.71238898038 0.55381964728 0.384012771128
1.34639685154 4.71238898038 0.494769661606 0.384012727427
1.79519580205 4.71238898038 0.538911776671 0.384010921913
2.24399475256 4.71238898038 0.549431479672 0.384005707245
2.69279370308 4.71238898038 0.515548071819 0.384005598522
3.14159265359 4.71238898038 0.492887746831 0.384005594397
0.0 4.71238898038 0.549183963726 0.333634263959
0.448798950513 4.71238898038 0.754736347048

AttributeError: 'list' object has no attribute 'shape'